In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

class Annonce:
    def __init__(self, idAnnonce, version, annee, kilometrage, prix, telephone, isPro, description, cote):
        self.idAnnonce = idAnnonce
        self.version = version
        self.annee = annee
        self.kilometrage = kilometrage
        self.prix = prix
        self.telephone = telephone
        self.isPro = isPro
        self.description = description
        self.cote = cote
    
    def to_dict(self):
        return {
            #'idAnnonce': self.idAnnonce,
            'annee': self.annee,
            'kilometrage': self.kilometrage, 
            'prix': self.prix,
            'version': self.version,
            'isPro': self.isPro, 
            'telephone': self.telephone,
            'cote': self.cote
            #'description':self.description
            
        }

def getSoupFromURL(url, method='get', data={}):

    if method == 'get':
        res = requests.get(url)
    elif method == 'post':
        res = requests.post(url, data=data)
    else:
        return None

    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        return soup
    else:
        return None

def getVersion(descr):
    m = re.search("ZOE (\w+)", descr)
    if m == None:
        return ""
    else:
        return m.group(1)

def prettifyPrix(prix):
    prix = prix.replace(" ","").replace ("€","").strip()
    return prix

def prettifyKm(km):
    km = km.replace(" ","").replace ("KM","").strip()
    return km

def getPhone(descr):
    expression = r"0[0-9]([ .-]?[0-9]{2}){4}"
    m = re.search(expression, descr)
    if m == None:
        return ""
    else:
        return m.group(0)
    
def getDetail(urlItem, cpt):
    annonce = Annonce(0, 0, 0, 0, 0,0, 0, 0, 0)
    if urlItem != None:
        soupItem = getSoupFromURL(urlItem)
        if soupItem:
            b = soupItem.find_all(class_="line")
            for b0 in b:
                
                b1 = b0.find_all(class_="property")
                b1Prime = b0.find_all(class_="value")
                  
                for b2 in b1:
                    for b2Prime in b1Prime:
                        annonce.idAnnonce = cpt
                        if (b2.text).upper() == "PRIX":
                            annonce.prix = prettifyPrix(b2Prime.text)
                        if (b2.text).upper() == "ANNÉE-MODÈLE":
                            annonce.annee = b2Prime.text.strip()
                        if (b2.text).upper() == "KILOMÉTRAGE":
                            annonce.kilometrage = prettifyKm(b2Prime.text)
                        if (b2.text).upper() == "DESCRIPTION :":
                            annonce.description = b2Prime.text.upper()
                            annonce.version = getVersion(annonce.description)
                            annonce.telephone = getPhone(annonce.description)
                        
                                               
    return annonce

def getQuoteValue(url):
    soup = getSoupFromURL(url)
    if soup:
        a = soup.find(class_="jsRefinedQuot")
        if a != None:
            return a.text
        else:
            return "2"
    else:
        
        return "-1"

def getCote(annee, version):
    url = "https://www.lacentrale.fr/cote-voitures-renault-zoe--" + str(annee) + "-.html"
    soup = getSoupFromURL(url)
    price = "0"
    if soup:
        a = soup.find_all(class_="listingResultLine auto")
        #print (a)
        for a0 in a:
        
            if version != None:
                for a1 in a0.find("h3"):
                    if version.upper() in a1:
                        # recup de la cote
                        urlItem = "https://www.lacentrale.fr/" + a0.find("a")["href"]
                        price = getQuoteValue(urlItem)
                        break
    return price
            

def getInfoVoitures(url):
    list = []
    soup = getSoupFromURL(url)
    if soup:
        a = soup.find_all(class_="tabsContent")
        for a0 in a:
            a1 = a0.find_all("li")
            cpt = 0
            for a2 in a1:
                urlItem = "http:" + a2.find ("a")["href"]
                
                annonce = getDetail(urlItem, cpt)  
                #print(annonce.annee, annonce.version)
                annonce.cote = getCote(annonce.annee, annonce.version)
                
                if a2.find(class_="ispro") != None:
                    annonce.isPro = a2.find(class_="ispro").text.replace("(","").replace(")","")
                else:
                    annonce.isPro = "NonPro"
                cpt = cpt + 1
                list.append(annonce)
            
    else:
        0
    return list

# essayer de paralleliser !!!

i = 1
df = pd.DataFrame()
while True :
    print(i)
    url = "https://www.leboncoin.fr/voitures/offres/ile_de_france/?o=" + str(i) + "&q=renault%20zoe"
    i = i + 1
    
    try :
        list = getInfoVoitures(url)
        if len(list) > 0:
            df = df.append(pd.DataFrame.from_records([s.to_dict() for s in list]))
        else:
            break
    except StopIteration :
        break

#print (df)
df.to_csv(r'bonCoin.csv', header=["Annee", "pro", "KM", "Prix", "Telephone", "Version", "Cote"], 
          index=None, sep=',', mode='a')



1
2
3
4
   annee    cote   isPro kilometrage   prix   telephone  version
0   2017       0  NonPro       20000  15900                   40
1   2015  10 250     pro       15254  10990               INTENS
2   2014   9 870     pro       28537   7900                 LIFE
3   2013   9 286     pro        8751   8990                 LIFE
4   2014  11 526  NonPro       29000   9700                     
5   2017       0  NonPro        4000  16800              BLANCHE
6   2017  16 075     pro         240  13990  0160657097     LIFE
7   2015  11 046     pro        7248  10690  0178455073     LIFE
8   2015  11 046     pro       11480  10300  0175448151     LIFE
9   2016  12 210  NonPro        5700   8900                 LIFE
10  2014  11 410     pro       34121  10790               INTENS
11  2015  10 250     pro        9474  10980               INTENS
12  2014  10 266     pro       10207  10290                 LIFE
13  2016  12 210     pro        9087  10480                 LIFE
14  2016  12 210 

In [6]:
print(df[df['annee']== '2013'])

   annee cote   isPro kilometrage   prix   telephone version
3   2013    0     pro        8751   8990                LIFE
15  2013    0     pro       17900   9490                 ZEN
17  2013    0  NonPro       27500   7800                LIFE
19  2013    0  NonPro       24000  10300                    
20  2013    0  NonPro       39500   8900              INTENS
22  2013    0     pro       23660   9200  0175448151     ZEN
23  2013    0  NonPro       19900   9100                  DE
27  2013    0     pro       13484   8500                LIFE
28  2013    0     pro       32671   9800  0174037256  INTENS
30  2013    0  NonPro       29500   7950              INTENS
31  2013    0     pro       31250   8480  0185470460     ZEN
32  2013    0     pro       31451   7990  0677372750    LIFE
33  2013    0     pro       70896   6790  0164572000    LIFE
2   2013    0  NonPro       55300   8000                 ZEN
6   2013    0     pro       81404   8700  0178455073     ZEN
19  2013    0     pro   

In [3]:
test = "<h3>INTENS CHARGE RAPIDE</h3>"
if "intens" in test:
    print ("ok")
else:
    print ("hjhjh")

hjhjh
